# Projet n°6 : Classifier automatiquement des biens de consommation - notebook n°3

**Consigne pour ce notebook :**

Pour cela, pourrais-tu tester la collecte de produits à base de “champagne” via l’API disponible ici ? 

Lien vers le site en question : https://developer.edamam.com/food-database-api 

Pourrais-tu ensuite nous proposer un script ou notebook Python permettant une extraction des 10 premiers produits dans un fichier “.csv”, contenant pour chaque produit les données suivantes : foodId, label, category, foodContentsLabel, image

**Stratégie de la collecte de données via l'API :**

1. **Récupération des données** : Utiliser l'API Edamam Food Database pour obtenir des informations alimentaires en fonction d'une requête spécifique dans notre situation les données liées au champagne.
2. **Extraction des informations pertinentes** : Filtrer et structurer les données brutes obtenues de l'API pour extraire uniquement les informations pertinentes.
3. **Sauvegarde des données dans un fichier CSV** : Stocker les informations extraites dans un fichier CSV 
4. **Configuration et exécution** : Définir les paramètres nécessaires (app_id, app_key, query)

Il y a également en parallèle la création d'un compte dédié sur le site Edamam, avec la création de l'API souhaitée sur le recueil "Food Database" qui nous permet de récupérer les informations sur l'APP_id & l'APP_key.


**Respect du RGPD :**

Pour rappel : les 5 principes fondamentaux du RGPD. 
- Transparence
- Limitation des finalités
- Minimisation des données
- Exactitude
- Limitation de conservation

Nos données proviennent d'un site dédié aux partages des données via API, où nous avons créé un compte sur celui-ci pour pouvoir disposer des données, sans par exemple faire du 'scrapping' avec BeautifulSoup, qui dans cette situation se passe d'autorisation ou autre pour collecter les données.
De plus, nos données ne sont pas sensibles et ne concernent pas direcement des individus, l'utilisation de ces données ne posent pas directement de problème tant que les sources par la suite lorsque nous les utilisons à notre compte.

In [19]:
# Importer les librairies nécessaires pour ce script
import requests
import csv
import pandas as pd

# API vers le site : Edamam

In [20]:
def get_food_data(query: str, app_id: str, app_key: str) -> dict:
    """
    Fonction pour obtenir les données alimentaires depuis l'API Edamam.
    
    Arguments:
    query -- La chaîne de recherche pour l'ingrédient (ici dans notre situation 'champagne').
    app_id -- L'ID de l'application Edamam.
    app_key -- La clé de l'application Edamam.
    
    Retourne:
    Un dictionnaire contenant les données de réponse de l'API.
    """
    url = "https://api.edamam.com/api/food-database/v2/parser"
    params = {
        'ingr': query,
        'app_id': app_id,
        'app_key': app_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erreur: {response.status_code}")
        return None

def extract_food_info(raw_data: dict, limit: int = 10) -> list:
    """
    Fonction pour extraire les informations alimentaires pertinentes depuis les données brutes.
    
    Arguments:
    raw_data -- Le dictionnaire de données brutes retourné par l'API.
    limit -- Le nombre maximal de produits à extraire (par défaut 10).
    
    Retourne:
    Une liste de dictionnaires contenant les informations des produits.
    """
    foods = []
    
    # Extraction des aliments dans la section 'parsed'
    for item in raw_data.get('parsed', []):
        if len(foods) >= limit:
            break
        food = item['food']
        food_data = {
            'foodId': food.get('foodId', ''),
            'label': food.get('label', ''),
            'category': food.get('category', ''),
            'foodContentsLabel': food.get('foodContentsLabel', ''),
            'image': food.get('image', '')
        }
        foods.append(food_data)
    
    # Extraction des aliments dans la section 'hints'
    for item in raw_data.get('hints', []):
        if len(foods) >= limit:
            break
        food = item['food']
        food_data = {
            'foodId': food.get('foodId', ''),
            'label': food.get('label', ''),
            'category': food.get('category', ''),
            'foodContentsLabel': food.get('foodContentsLabel', ''),
            'image': food.get('image', '')
        }
        foods.append(food_data)
    
    return foods

def save_to_csv(data: list, filename: str = 'champagne_products.csv') -> None:
    """
    Fonction pour sauvegarder les données alimentaires dans un fichier CSV.
    
    Arguments:
    data -- Une liste de dictionnaires contenant les informations des produits.
    filename -- Le nom du fichier CSV à créer (par défaut 'champagne_products.csv').
    """
    if not data:
        print("Aucune donnée à sauvegarder.")
        return
    
    keys = data[0].keys()
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(data)
    print(f"Données sauvegardées dans {filename}")

# Configuration
app_id = 'd553ce75'
app_key = 'f4aa0a940217fc66c30ed770f5b6e85f'
query = 'champagne'

# Collecte des données
raw_data = get_food_data(query, app_id, app_key)
if raw_data:
    foods = extract_food_info(raw_data, limit=10)
    filename = 'champagne_products.csv'
    save_to_csv(foods, filename)
else:
    print("Échec de la récupération des données")


Données sauvegardées dans champagne_products.csv


# Afficher le résultat du fichier CSV

In [18]:
# Afficher les résultats stockés dans le fichier csv
df = pd.read_csv('champagne_products.csv')
df

,foodId,label,category,foodContentsLabel,image
0,food_a656mk2a5dmqb2adiamu6beihduu,Champagne,Generic foods,NaN,https://www.edamam.com/food-img/a71/a718cf3c52...
1,food_a656mk2a5dmqb2adiamu6beihduu,Champagne,Generic foods,NaN,https://www.edamam.com/food-img/a71/a718cf3c52...
2,food_b753ithamdb8psbt0w2k9aquo06c,"Champagne Vinaigrette, Champagne",Packaged foods,OLIVE OIL; BALSAMIC VINEGAR; CHAMPAGNE VINEGAR...,NaN
3,food_b3dyababjo54xobm6r8jzbghjgqe,"Champagne Vinaigrette, Champagne",Packaged foods,INGREDIENTS: WATER; CANOLA OIL; CHAMPAGNE VINE...,https://www.edamam.com/food-img/d88/d88b64d973...
4,food_a9e0ghsamvoc45bwa2ybsa3gken9,"Champagne Vinaigrette, Champagne",Packaged foods,CANOLA AND SOYBEAN OIL; WHITE WINE (CONTAINS S...,NaN
5,food_an4jjueaucpus2a3u1ni8auhe7q9,"Champagne Vinaigrette, Champagne",Packaged foods,WATER; CANOLA AND SOYBEAN OIL; WHITE WINE (CON...,NaN
6,food_bmu5dmkazwuvpaa5prh1daa8jxs0,"Champagne Dressing, Champagne",Packaged foods,SOYBEAN OIL; WHITE WINE (PRESERVED WITH SULFIT...,https://www.edamam.com/food-img/ab2/ab2459fc2a...
7,food_alpl44taoyv11ra0lic1qa8xculi,Champagne Buttercream,Generic meals,sugar; butter; shortening; vanilla; champagne;...,NaN
8,food_am5egz6aq3fpjlaf8xpkdbc2asis,Champagne Truffles,Generic meals,butter; cocoa; sweetened condensed milk; vanil...,NaN
9,food_bcz8rhiajk1fuva0vkfmeakbouc0,Champagne Vinaigrette,Generic meals,champagne vinegar; olive oil; Dijon mustard; s...,NaN
